In [48]:
import os
from datetime import timedelta, datetime
import talib
import pandas as pd
from sqlalchemy import create_engine
import dotenv

In [49]:
dotenv.load_dotenv()
SQLALCHEMY_DATABASE_URI = os.getenv('SQLALCHEMY_DATABASE_URI')

In [50]:
db_engine = create_engine(
    SQLALCHEMY_DATABASE_URI, 
    pool_size=10, 
    max_overflow=20,
    )

In [51]:
request_dict = {
    'symbol': 'KOTAKBANK',
    'dates': {
        'start': '2018-01-01',
        'end': '2019-01-01'
    }
}

In [52]:
df = pd.read_sql('''
    SELECT *
    FROM stock_daily_data
    WHERE symbol = '{}'
    AND   TIMESTAMP BETWEEN '{}' AND '{}'
    ORDER BY TIMESTAMP
    LIMIT 1000;
    '''.format(
            request_dict['symbol'],
            datetime.strptime(request_dict['dates']['start'], '%Y-%m-%d') - timedelta(days=500),
            request_dict['dates']['end'],
        ), db_engine, index_col=['timestamp']);

In [53]:
'''
# Generate CSV
df.to_csv('CSV/{}_{}_{}.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))
'''
# Preview
df_extended = df.copy(deep=True)
df_extended

,Open,High,Low,Close,Volume,symbol,updated_at
timestamp,,,,,,,
2016-08-19,785.95,785.95,774.00,778.75,996170,KOTAKBANK,2020-04-16 23:54:18
2016-08-22,780.00,782.00,768.80,772.85,612762,KOTAKBANK,2020-04-16 23:54:18
2016-08-23,773.35,780.00,771.05,778.35,572986,KOTAKBANK,2020-04-16 23:54:18
2016-08-24,779.00,781.90,772.10,777.00,1141719,KOTAKBANK,2020-04-16 23:54:18
2016-08-25,778.00,784.00,772.05,780.75,1477112,KOTAKBANK,2020-04-16 23:54:18
...,...,...,...,...,...,...,...
2018-12-26,1234.50,1250.80,1210.50,1248.15,2379224,KOTAKBANK,2020-04-16 23:54:18
2018-12-27,1257.25,1258.65,1235.35,1240.25,2653738,KOTAKBANK,2020-04-16 23:54:18
2018-12-28,1244.25,1250.90,1235.05,1241.35,1740134,KOTAKBANK,2020-04-16 23:54:18


In [54]:
# calculating indicators now
df_extended['EMA50'] = talib.EMA(df_extended.Close, timeperiod=50) 
df_extended['EMA200'] = talib.EMA(df_extended.Close, timeperiod=200)
df_extended['rsa14'] = talib.RSI(df_extended.Close, timeperiod=14)
df_extended['sar'] = talib.SAR(
    df_extended.High,
    df_extended.Low,
    acceleration=0.02, 
    maximum=0.2,
    )
df_extended['macd'], df_extended['macdsignal'], df_extended['macdhist'] = talib.MACDEXT(
        df_extended.Close, 
        fastperiod=12, 
        fastmatype=0, 
        slowperiod=26, 
        slowmatype=0, 
        signalperiod=9, 
        signalmatype=0,
        )
df_extended['slowk'], df_extended['slowkd'] = talib.STOCH(
        df_extended.High, df_extended.Low, df_extended.Close,
        fastk_period=14, 
        slowk_period=3, 
        slowk_matype=0, 
        slowd_period=3, 
        slowd_matype=0,
        )
df_extended['upperband'], df_extended['middleband'], df_extended['lowerband'] = talib.BBANDS(
        df_extended.Close, 
        timeperiod=5, 
        nbdevup=2, 
        nbdevdn=2, 
        matype=0,
        )

In [55]:
# Preview
df_extended

,Open,High,Low,Close,Volume,symbol,updated_at,EMA50,EMA200,rsa14,sar,macd,macdsignal,macdhist,slowk,slowkd,upperband,middleband,lowerband
timestamp,,,,,,,,,,,,,,,,,,,
2016-08-19,785.95,785.95,774.00,778.75,996170,KOTAKBANK,2020-04-16 23:54:18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-22,780.00,782.00,768.80,772.85,612762,KOTAKBANK,2020-04-16 23:54:18,NaN,NaN,NaN,785.950000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-23,773.35,780.00,771.05,778.35,572986,KOTAKBANK,2020-04-16 23:54:18,NaN,NaN,NaN,785.607000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-24,779.00,781.90,772.10,777.00,1141719,KOTAKBANK,2020-04-16 23:54:18,NaN,NaN,NaN,785.270860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-08-25,778.00,784.00,772.05,780.75,1477112,KOTAKBANK,2020-04-16 23:54:18,NaN,NaN,NaN,784.941443,NaN,NaN,NaN,NaN,NaN,782.809687,777.54,772.270313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-26,1234.50,1250.80,1210.50,1248.15,2379224,KOTAKBANK,2020-04-16 23:54:18,1204.026654,1188.128760,56.499640,1313.946888,19.679487,28.566382,-8.886895,38.268077,35.946778,1251.530632,1235.16,1218.789368
2018-12-27,1257.25,1258.65,1235.35,1240.25,2653738,KOTAKBANK,2020-04-16 23:54:18,1205.447177,1188.647380,54.793558,1311.087950,20.671154,27.111574,-6.440420,40.754501,38.344289,1251.580607,1238.08,1224.579393
2018-12-28,1244.25,1250.90,1235.05,1241.35,1740134,KOTAKBANK,2020-04-16 23:54:18,1206.855131,1189.171784,54.997330,1308.286191,19.812500,25.233583,-5.421083,47.616140,42.212906,1252.662236,1239.09,1225.517764


In [56]:
# Generate CSV
df_extended.to_csv('CSV/{}_{}_{}_indicators.csv'.format(
    request_dict['symbol'],
    request_dict['dates']['start'],
    request_dict['dates']['end'],
))

In [60]:
n_days_lst = [5, 10, ]
p_target = 0.10
p_stoploss = -0.10

In [61]:
def get_true_signal(change_n):
    if change_n >= p_target:
        return 'BUY'
    elif change_n <= p_stoploss:
        return 'SELL'
    else:
        return 'NEUTRAL'
    

In [62]:
df_close_plus = df.copy(deep=True).drop(columns=['updated_at'])
for n_days in n_days_lst:
    df_close_plus['Close_{}'.format(n_days)] = df_close_plus['Close'].shift(-n_days)
    df_close_plus['Change_{}'.format(n_days)] = (
        (df_close_plus['Close_{}'.format(n_days)] - df_close_plus['Close'])
    )/df_close_plus['Close']


    df_close_plus['Signal_{}'.format(n_days)] = df_close_plus['Change_{}'.format(n_days)].apply(get_true_signal)
# df_close_plus


with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df_close_plus)

,Open,High,Low,Close,Volume,symbol,Close_5,Change_5,Signal_5,Close_10,Change_10,Signal_10
timestamp,,,,,,,,,,,,
2016-08-19,785.95,785.95,774.00,778.75,996170,KOTAKBANK,780.80,0.002632,NEUTRAL,813.60,0.044751,NEUTRAL
2016-08-22,780.00,782.00,768.80,772.85,612762,KOTAKBANK,781.40,0.011063,NEUTRAL,818.05,0.058485,NEUTRAL
2016-08-23,773.35,780.00,771.05,778.35,572986,KOTAKBANK,786.15,0.010021,NEUTRAL,804.10,0.033083,NEUTRAL
2016-08-24,779.00,781.90,772.10,777.00,1141719,KOTAKBANK,806.90,0.038481,NEUTRAL,819.75,0.055019,NEUTRAL
2016-08-25,778.00,784.00,772.05,780.75,1477112,KOTAKBANK,810.70,0.038361,NEUTRAL,818.85,0.048799,NEUTRAL
2016-08-26,782.50,789.00,772.85,780.80,853973,KOTAKBANK,813.60,0.042008,NEUTRAL,813.30,0.041624,NEUTRAL
2016-08-29,780.70,788.90,775.15,781.40,762039,KOTAKBANK,818.05,0.046903,NEUTRAL,805.95,0.031418,NEUTRAL
2016-08-30,781.90,787.05,780.20,786.15,799025,KOTAKBANK,804.10,0.022833,NEUTRAL,808.60,0.028557,NEUTRAL
2016-08-31,795.00,814.95,793.15,806.90,3346444,KOTAKBANK,819.75,0.015925,NEUTRAL,806.60,-0.000372,NEUTRAL
